<a href="https://colab.research.google.com/github/TiFcode/TiFai/blob/main/TiFai_tif3_object_recognition_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print('*** START ********************************************************')

!pwd
!ls

outputDetails = False
size = (224, 224)
googleDriveFolderPath = '/content/drive/My Drive/Personal/Dev/ChatGPT/2023-01-05 - AI - object recognition/'

In [ ]:
from google.colab.patches import cv2_imshow

def display_image(image, description):
  print(f'{description}:')
  print(f'Image size: {image.shape}')
  cv2_imshow(image)


In [ ]:
import inspect

def retrieve_variable_name(var):
    callers_local_vars = inspect.currentframe().f_back.f_locals.items()
    return [var_name for var_name, var_val in callers_local_vars if var_val is var]
    
def retrieve_variable_name_from_call(var):
    callers_local_vars = inspect.currentframe().f_back.f_back.f_locals.items()
    return [var_name for var_name, var_val in callers_local_vars if var_val is var]

def printVar(variable):
  print(f'{retrieve_variable_name_from_call(variable)} is [{variable}]')

def printVarDesc(variable, description):
  print(f'{description} is [{variable}]')

In [ ]:
from google.colab import drive
import glob

# Mount Google Drive
drive.mount('/content/drive')
!pwd
!ls

# Search for a file by name
#pattern = '/content/drive/My Drive/**/*20230105_151957.jpg'
#filenames = glob.glob(pattern, recursive=True)
#print(filenames)


In [ ]:
import os
import cv2

def loadAndProcessAllFilesFromFolder(folderPath):
  filenames = os.listdir(folderPath)

  imagesArray = []
  for filename in filenames:
    filenameFullPath = folderPath + filename
    if outputDetails:
      print(f'Found file [{filename}] with size [{os.stat(filenameFullPath).st_size}] bytes.')
    
    image = cv2.imread(filenameFullPath)
    if outputDetails:
      display_image(image, filenameFullPath)
    imagesArray.append(image)

    if outputDetails:
      print(f'imagesArray contains [{len(imagesArray)}] elements.')

  # Resize the images to a fixed size
  imagesArray = [cv2.resize(image, size) for image in imagesArray]

  # Convert the images to a format that can be used as input to a CNN
  imagesArray = [cv2.cvtColor(image, cv2.COLOR_BGR2RGB) for image in imagesArray]

  return imagesArray

In [ ]:
print('Loading training images...')
training_images = loadAndProcessAllFilesFromFolder(googleDriveFolderPath + 'training_images/')
print(f'training_images contains [{len(training_images)}] images.')

print('Loading validation images...')
validation_images = loadAndProcessAllFilesFromFolder(googleDriveFolderPath + 'validation_images/')
print(f'validation_images contains [{len(validation_images)}] images.')

print('Loading search images...')
search_images = loadAndProcessAllFilesFromFolder(googleDriveFolderPath + 'search_images/')
print(f'search_images contains [{len(search_images)}] images.')

print('*** FINISH LOADING IMAGES ********************************************************')


In [ ]:
#from sklearn.model_selection import train_test_split

# Split the object images into a training set and a validation set
#X_train, X_val = train_test_split(object_images, test_size=0.2, random_state=42)


In [ ]:
# Convert the lists to numpy arrays
printVarDesc(type(training_images), "type(training_images)")
printVarDesc(len(training_images), "len(training_images)")
training_images = np.array(training_images)
printVarDesc(type(training_images), "type(training_images)")
printVarDesc(training_images.shape[0], "training_images.shape[0]")

validation_images = np.array(validation_images)

search_images = np.array(search_images)

# Reshape the arrays

training_batch_size = training_images.shape[0]
printVar(training_batch_size)
training_images = np.reshape(training_images, (training_batch_size, 224, 224, 3))
printVarDesc(training_images.shape, "training_images.shape")

validation_batch_size = validation_images.shape[0]
printVar(validation_batch_size)
validation_images = np.reshape(validation_images, (validation_batch_size, 224, 224, 3))
printVarDesc(validation_images.shape, "validation_images.shape")

# Create the target data
target_data = np.ones(len(training_images))

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.models import Sequential
import numpy as np

# Create the model
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(training_images, target_data, batch_size=training_batch_size, epochs=10, validation_data=(validation_images,))



